In [1]:
import numpy as np
from scipy.optimize import fsolve
import pandas as pd

In [ ]:
# Given molalities in mol/kg
molalities = np.array([0.10, 0.15, 0.20, 0.25, 0.30, 0.35, 0.40, 0.45, 0.50, 0.55, 0.60, 0.70, 0.80, 0.90, 1.00, 1.10, 1.20, 1.23])

In [3]:
# Equilibrium constants at 25°C ka values from pka = -log(ka)
Ka_NH4 = 10**-9.25  # NH4+ <=> NH3 + H+ 
Ka1_H3PO4 = 10**-2.1  # H3PO4 <=> H2PO4- + H+
Ka2_H2PO4 = 10**-7.2  # H2PO4- <=> HPO4^2- + H+
Ka3_HPO4 = 10**-12.3  # HPO4^2- <=> PO4^3- + H+ 
Kw = 10**-14 # Water ionization constant

In [ ]:
def pH_NH4PO4(molality_list, initial_guess=7.0):
    """
    Solve for the pH of (NH4)3PO4 solutions for given molality (m).
    """

    def charge_balance(pH_array, m):
        """
        Function to obtain charge balance equation residual: cations - anions
        """
        pH = pH_array[0]
        H_conc = 10.0**(-pH)
        if H_conc <= 0:
            return [1e10]

        OH_conc = Kw / H_conc
        
        # total ammonia = 3*m because (NH4)3PO4 has 3 NH4+ per PO4
        NH4_conc = (3.0*m) / (1.0 + (Ka_NH4 / H_conc))
        NH3_conc = NH4_conc * (Ka_NH4 / H_conc)

        # total phosphate = m. [H2PO4-] equilibrium equation will be the total phosphate concentration/denom
        denom = (
            1.0
            + (H_conc / Ka1_H3PO4)
            + (Ka2_H2PO4 / H_conc)
            + ((Ka2_H2PO4 * Ka3_HPO4) / (H_conc * H_conc))
        )
        H2PO4_conc = m / denom
        HPO4_conc = H2PO4_conc * (Ka2_H2PO4 / H_conc)
        PO4_conc = H2PO4_conc * ((Ka2_H2PO4*Ka3_HPO4)/(H_conc*H_conc))

        # cations - anions
        diff = (H_conc + NH4_conc) - (OH_conc + H2PO4_conc + 2.0*HPO4_conc + 3.0*PO4_conc)
        return [diff]

    results = []

    for m in molality_list:
        # Solve for pH by finding root with fsolve
        sol = fsolve(charge_balance, [initial_guess], args=(m,))
        pH = sol[0]

        #Recalculate concentrations of components for checks
        H_conc = 10.0**(-pH)
        OH_conc = Kw / H_conc

        NH4_conc = (3.0*m) / (1.0 + (Ka_NH4 / H_conc))
        NH3_conc = NH4_conc * (Ka_NH4 / H_conc)

        denom = (
            1.0
            + (H_conc / Ka1_H3PO4)
            + (Ka2_H2PO4 / H_conc)
            + ((Ka2_H2PO4 * Ka3_HPO4) / (H_conc * H_conc))
        )
        H2PO4_conc = m / denom
        HPO4_conc = H2PO4_conc * (Ka2_H2PO4 / H_conc)
        PO4_conc = H2PO4_conc * ((Ka2_H2PO4*Ka3_HPO4)/(H_conc*H_conc))
        H3PO4_conc = H2PO4_conc * (H_conc / Ka1_H3PO4)

        # Charge balance check should result close to 0
        residual = charge_balance([pH], m)[0]

        # Mass balance checks
        phosphate_sum = H3PO4_conc + H2PO4_conc + HPO4_conc + PO4_conc
        ammonia_sum = NH4_conc + NH3_conc  # should be ~ 3*m

        results.append({
            "Molality": m,
            "pH": pH,
            "ChargeBalanceResidual": residual,
            "PhosphateSum": phosphate_sum,
            "AmmoniaSum": ammonia_sum,
            "[H+]": H_conc,
            "[OH-]": OH_conc,
            "[NH4+]": NH4_conc,
            "[NH3]": NH3_conc,
            "[H3PO4]": H3PO4_conc,
            "[H2PO4-]": H2PO4_conc,
            "[HPO4^2-]": HPO4_conc,
            "[PO4^3-]": PO4_conc
        })

    return pd.DataFrame(results)

df_NH4PO4 = pH_NH4PO4(molalities, initial_guess=7.0)

In [10]:
df_NH4PO4

,Molality,pH,ChargeBalanceResidual,PhosphateSum,AmmoniaSum,[H+],[OH-],[NH4+],[NH3],[H3PO4],[H2PO4-],[HPO4^2-],[PO4^3-]
0,0.10,8.964537,-1.665335e-16,0.10,0.30,1.085083e-09,0.000009,0.198360,0.101640,2.589900e-10,0.001690,0.098270,0.000040
1,0.15,8.964556,-5.551115e-17,0.15,0.45,1.085035e-09,0.000009,0.297535,0.152465,3.884507e-10,0.002535,0.147405,0.000061
2,0.20,8.964566,0.000000e+00,0.20,0.60,1.085011e-09,0.000009,0.396711,0.203289,5.179113e-10,0.003380,0.196540,0.000081
3,0.25,8.964572,1.110223e-16,0.25,0.75,1.084996e-09,0.000009,0.495886,0.254114,6.473720e-10,0.004224,0.245674,0.000101
4,0.30,8.964576,-4.440892e-16,0.30,0.90,1.084987e-09,0.000009,0.595062,0.304938,7.768327e-10,0.005069,0.294809,0.000121
5,0.35,8.964578,3.330669e-16,0.35,1.05,1.084980e-09,0.000009,0.694237,0.355763,9.062933e-10,0.005914,0.343944,0.000142
6,0.40,8.964580,1.110223e-16,0.40,1.20,1.084975e-09,0.000009,0.793412,0.406588,1.035754e-09,0.006759,0.393079,0.000162
7,0.45,8.964582,4.440892e-16,0.45,1.35,1.084971e-09,0.000009,0.892588,0.457412,1.165215e-09,0.007604,0.442214,0.000182
8,0.50,8.964583,-2.220446e-16,0.50,1.50,1.084967e-09,0.000009,0.991763,0.508237,1.294675e-09,0.008448,0.491349,0.000202
9,0.55,8.964584,2.220446e-16,0.55,1.65,1.084965e-09,0.000009,1.090939,0.559061,1.424136e-09,0.009293,0.540484,0.000223


In [ ]:
def pH_NH4H2PO4(molality_list, initial_guess=7.0):
    """
    Solve for the pH of NH4H2PO4 solutions for given molality (m).
    """

    def charge_balance(pH_array, m):
        """
        Function to obtain charge balance equation residual: cations - anions
        """
        pH = pH_array[0]
        H_conc = 10.0**(-pH)
        if H_conc <= 0:
            return [1e10]

        OH_conc = Kw / H_conc
        

        NH4_conc = m / (1.0 + (Ka_NH4 / H_conc))
        NH3_conc = NH4_conc * (Ka_NH4 / H_conc)

        # total phosphate = m. [H2PO4-] equilibrium equation will be the total phosphate concentration/denom
        denom = (
            1.0
            + (H_conc / Ka1_H3PO4)
            + (Ka2_H2PO4 / H_conc)
            + ((Ka2_H2PO4 * Ka3_HPO4) / (H_conc * H_conc))
        )
        H2PO4_conc = m / denom
        HPO4_conc = H2PO4_conc * (Ka2_H2PO4 / H_conc)
        PO4_conc = H2PO4_conc * ((Ka2_H2PO4*Ka3_HPO4)/(H_conc*H_conc))

        # cations - anions
        diff = (H_conc + NH4_conc) - (OH_conc + H2PO4_conc + 2.0*HPO4_conc + 3.0*PO4_conc)
        return [diff]

    results = []

    for m in molality_list:
        # Solve for pH by finding root with fsolve
        sol = fsolve(charge_balance, [initial_guess], args=(m,))
        pH = sol[0]

        #Recalculate concentrations of components for checks
        H_conc = 10.0**(-pH)
        OH_conc = Kw / H_conc

        NH4_conc = m / (1.0 + (Ka_NH4 / H_conc))
        NH3_conc = NH4_conc * (Ka_NH4 / H_conc)

        denom = (
            1.0
            + (H_conc / Ka1_H3PO4)
            + (Ka2_H2PO4 / H_conc)
            + ((Ka2_H2PO4 * Ka3_HPO4) / (H_conc * H_conc))
        )
        H2PO4_conc = m / denom
        HPO4_conc = H2PO4_conc * (Ka2_H2PO4 / H_conc)
        PO4_conc = H2PO4_conc * ((Ka2_H2PO4*Ka3_HPO4)/(H_conc*H_conc))
        H3PO4_conc = H2PO4_conc * (H_conc / Ka1_H3PO4)

        # Charge balance check should result close to 0
        residual = charge_balance([pH], m)[0]

        # Mass balance checks
        phosphate_sum = H3PO4_conc + H2PO4_conc + HPO4_conc + PO4_conc
        ammonia_sum = NH4_conc + NH3_conc 

        results.append({
            "Molality": m,
            "pH": pH,
            "ChargeBalanceResidual": residual,
            "PhosphateSum": phosphate_sum,
            "AmmoniaSum": ammonia_sum,
            "[H+]": H_conc,
            "[OH-]": OH_conc,
            "[NH4+]": NH4_conc,
            "[NH3]": NH3_conc,
            "[H3PO4]": H3PO4_conc,
            "[H2PO4-]": H2PO4_conc,
            "[HPO4^2-]": HPO4_conc,
            "[PO4^3-]": PO4_conc
        })

    return pd.DataFrame(results)

df_NH4H2PO4 = pH_NH4H2PO4(molalities, initial_guess=7.0)

In [16]:
df_NH4H2PO4

,Molality,pH,ChargeBalanceResidual,PhosphateSum,AmmoniaSum,[H+],[OH-],[NH4+],[NH3],[H3PO4],[H2PO4-],[HPO4^2-],[PO4^3-]
0,0.10,4.687992,0.000000e+00,0.10,0.10,0.000021,4.875195e-10,0.099997,0.000003,0.000288,0.099406,0.000306,6.663990e-12
1,0.15,4.683125,0.000000e+00,0.15,0.15,0.000021,4.820870e-10,0.149996,0.000004,0.000437,0.149110,0.000454,9.774470e-12
2,0.20,4.680651,0.000000e+00,0.20,0.20,0.000021,4.793477e-10,0.199995,0.000005,0.000586,0.198813,0.000601,1.288495e-11
3,0.25,4.679152,0.000000e+00,0.25,0.25,0.000021,4.776966e-10,0.249993,0.000007,0.000735,0.248516,0.000749,1.599542e-11
4,0.30,4.678147,-1.110223e-16,0.30,0.30,0.000021,4.765926e-10,0.299992,0.000008,0.000884,0.298219,0.000897,1.910589e-11
5,0.35,4.677427,-1.110223e-16,0.35,0.35,0.000021,4.758025e-10,0.349991,0.000009,0.001033,0.347923,0.001045,2.221637e-11
6,0.40,4.676885,0.000000e+00,0.40,0.40,0.000021,4.752091e-10,0.399989,0.000011,0.001182,0.397626,0.001192,2.532684e-11
7,0.45,4.676462,5.551115e-17,0.45,0.45,0.000021,4.747470e-10,0.449988,0.000012,0.001331,0.447329,0.001340,2.843731e-11
8,0.50,4.676124,-1.110223e-16,0.50,0.50,0.000021,4.743771e-10,0.499987,0.000013,0.001480,0.497032,0.001488,3.154779e-11
9,0.55,4.675846,0.000000e+00,0.55,0.55,0.000021,4.740741e-10,0.549986,0.000014,0.001629,0.546736,0.001636,3.465826e-11


In [ ]:
def pH_NaH2PO4(molality_list, initial_guess=7.0):
    """
    Solve for the pH of NaH2PO4 solutions for given molality (m).
    """

    def charge_balance(pH_array, m):
        """
        Function to obtain charge balance equation residual: cations - anions
        """
        pH = pH_array[0]
        H_conc = 10.0**(-pH)
        if H_conc <= 0:
            return [1e10]

        OH_conc = Kw / H_conc
        
        Na_conc = m

        # total phosphate = m. [H2PO4-] equilibrium equation will be the total phosphate concentration/denom
        denom = (
            1.0
            + (H_conc / Ka1_H3PO4)
            + (Ka2_H2PO4 / H_conc)
            + ((Ka2_H2PO4 * Ka3_HPO4) / (H_conc * H_conc))
        )
        H2PO4_conc = m / denom
        HPO4_conc = H2PO4_conc * (Ka2_H2PO4 / H_conc)
        PO4_conc = H2PO4_conc * ((Ka2_H2PO4*Ka3_HPO4)/(H_conc*H_conc))

        # cations - anions
        diff = (H_conc + Na_conc) - (OH_conc + 3 * PO4_conc + 2 * HPO4_conc + H2PO4_conc)
        return [diff]

    results = []

    for m in molality_list:
        # Solve for pH by finding root with fsolve
        sol = fsolve(charge_balance, [initial_guess], args=(m,))
        pH = sol[0]

        #Recalculate concentrations of components for checks
        H_conc = 10.0**(-pH)
        OH_conc = Kw / H_conc

        Na_conc = m

        denom = (
            1.0
            + (H_conc / Ka1_H3PO4)
            + (Ka2_H2PO4 / H_conc)
            + ((Ka2_H2PO4 * Ka3_HPO4) / (H_conc * H_conc))
        )
        H2PO4_conc = m / denom
        HPO4_conc = H2PO4_conc * (Ka2_H2PO4 / H_conc)
        PO4_conc = H2PO4_conc * ((Ka2_H2PO4*Ka3_HPO4)/(H_conc*H_conc))
        H3PO4_conc = H2PO4_conc * (H_conc / Ka1_H3PO4)

        # Charge balance check should result close to 0
        residual = charge_balance([pH], m)[0]

        # Mass balance checks
        phosphate_sum = H3PO4_conc + H2PO4_conc + HPO4_conc + PO4_conc
        sodium_sum = Na_conc 

        results.append({
            "Molality": m,
            "pH": pH,
            "ChargeBalanceResidual": residual,
            "PhosphateSum": phosphate_sum,
            "SodiumSum": sodium_sum,
            "[H+]": H_conc,
            "[OH-]": OH_conc,
            "[Na+]": Na_conc,
            "[H3PO4]": H3PO4_conc,
            "[H2PO4-]": H2PO4_conc,
            "[HPO4^2-]": HPO4_conc,
            "[PO4^3-]": PO4_conc
        })

    return pd.DataFrame(results)

df_NaH2PO4 = pH_NaH2PO4(molalities, initial_guess=7.0)

In [14]:
df_NaH2PO4

,Molality,pH,ChargeBalanceResidual,PhosphateSum,SodiumSum,[H+],[OH-],[Na+],[H3PO4],[H2PO4-],[HPO4^2-],[PO4^3-]
0,0.10,4.689908,0.000000e+00,0.10,0.10,0.000020,4.896754e-10,0.10,0.000287,0.099406,0.000307,6.723053e-12
1,0.15,4.685042,0.000000e+00,0.15,0.15,0.000021,4.842189e-10,0.15,0.000435,0.149109,0.000456,9.861104e-12
2,0.20,4.682567,0.000000e+00,0.20,0.20,0.000021,4.814674e-10,0.20,0.000583,0.198813,0.000604,1.299915e-11
3,0.25,4.681068,0.000000e+00,0.25,0.25,0.000021,4.798090e-10,0.25,0.000732,0.248516,0.000752,1.613720e-11
4,0.30,4.680064,5.551115e-17,0.30,0.30,0.000021,4.787002e-10,0.30,0.000880,0.298219,0.000901,1.927524e-11
5,0.35,4.679343,0.000000e+00,0.35,0.35,0.000021,4.779066e-10,0.35,0.001028,0.347923,0.001049,2.241328e-11
6,0.40,4.678801,-1.110223e-16,0.40,0.40,0.000021,4.773105e-10,0.40,0.001177,0.397626,0.001198,2.555133e-11
7,0.45,4.678379,0.000000e+00,0.45,0.45,0.000021,4.768464e-10,0.45,0.001325,0.447329,0.001346,2.868937e-11
8,0.50,4.678040,-1.110223e-16,0.50,0.50,0.000021,4.764748e-10,0.50,0.001473,0.497032,0.001494,3.182742e-11
9,0.55,4.677763,-1.110223e-16,0.55,0.55,0.000021,4.761706e-10,0.55,0.001622,0.546736,0.001643,3.496546e-11
